## Introduksjon

Denne leksjonen vil ta for seg:
- Hva funksjonskall er og når det brukes
- Hvordan lage et funksjonskall med Azure OpenAI
- Hvordan integrere et funksjonskall i en applikasjon

## Læringsmål

Etter å ha fullført denne leksjonen vil du vite og forstå:

- Hensikten med å bruke funksjonskall
- Sette opp funksjonskall med Azure Open AI Service
- Designe effektive funksjonskall tilpasset din applikasjons behov


## Forstå funksjonskall

I denne leksjonen skal vi lage en funksjon for vår utdanningsstartup som lar brukere bruke en chatbot for å finne tekniske kurs. Vi vil anbefale kurs som passer til deres ferdighetsnivå, nåværende rolle og teknologi de er interessert i.

For å få til dette vil vi bruke en kombinasjon av:
 - `Azure Open AI` for å lage en chatopplevelse for brukeren
 - `Microsoft Learn Catalog API` for å hjelpe brukere å finne kurs basert på deres forespørsel
 - `Function Calling` for å ta brukerens spørsmål og sende det til en funksjon som gjør API-forespørselen.

La oss først se på hvorfor vi ønsker å bruke funksjonskall i det hele tatt:

print("Meldinger i neste forespørsel:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # få et nytt svar fra GPT hvor den kan se funksjonens respons


print(second_response.choices[0].message)


### Hvorfor bruke Function Calling

Hvis du har fullført noen av de andre leksjonene i dette kurset, forstår du sikkert hvor kraftige Large Language Models (LLMs) kan være. Forhåpentligvis har du også lagt merke til noen av begrensningene deres.

Function Calling er en funksjon i Azure Open AI Service som hjelper til med å overvinne følgende begrensninger:
1) Konsistent responsformat
2) Mulighet til å bruke data fra andre kilder i en applikasjon i en chatkontekst

Før function calling var svarene fra en LLM ustrukturerte og inkonsistente. Utviklere måtte skrive komplisert valideringskode for å kunne håndtere alle variasjoner i svarene.

Brukere kunne ikke få svar som "Hva er været i Stockholm akkurat nå?". Dette skyldes at modellene var begrenset til tidspunktet dataene ble trent på.

La oss se på eksempelet under som illustrerer dette problemet:

La oss si at vi ønsker å lage en database med studentdata slik at vi kan foreslå riktig kurs til dem. Under har vi to beskrivelser av studenter som inneholder veldig like data.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vi ønsker å sende dette til en LLM for å analysere dataene. Dette kan senere brukes i applikasjonen vår for å sende det til et API eller lagre det i en database.

La oss lage to identiske forespørsler der vi instruerer LLM om hvilken informasjon vi er interessert i:


Vi ønsker å sende dette til en LLM for å analysere delene som er viktige for vårt produkt. Slik kan vi lage to identiske instruksjoner for å veilede LLM-en:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Etter å ha laget disse to promptene, sender vi dem til LLM ved å bruke `openai.ChatCompletion`. Vi lagrer prompten i variabelen `messages` og tildeler rollen til `user`. Dette er for å etterligne en melding fra en bruker som skrives til en chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Selv om promptene er de samme og beskrivelsene ligner, kan vi få ulike formater på `Grades`-egenskapen.

Hvis du kjører cellen over flere ganger, kan formatet være `3.7` eller `3.7 GPA`.

Dette skjer fordi LLM-en tar inn ustrukturert data i form av den skrevne prompten og returnerer også ustrukturert data. Vi trenger et strukturert format slik at vi vet hva vi kan forvente når vi skal lagre eller bruke denne dataen.

Ved å bruke funksjonskall kan vi sørge for at vi får tilbake strukturert data. Når vi bruker funksjonskall, kjører eller utfører ikke LLM-en faktisk noen funksjoner. I stedet lager vi en struktur som LLM-en skal følge i svarene sine. Vi bruker så disse strukturerte svarene for å vite hvilken funksjon vi skal kjøre i applikasjonene våre.


### Bruksområder for å bruke funksjonskall

**Kalle eksterne verktøy**  
Chatboter er flinke til å gi svar på spørsmål fra brukere. Ved å bruke funksjonskall kan chatbotene bruke meldinger fra brukerne til å utføre bestemte oppgaver. For eksempel kan en student be chatboten om å "Sende e-post til læreren min og si at jeg trenger mer hjelp med dette emnet". Dette kan gjøre et funksjonskall til `send_email(to: string, body: string)`

**Lage API- eller databaseforespørsler**  
Brukere kan finne informasjon ved å bruke naturlig språk som blir omgjort til en formatert spørring eller API-forespørsel. Et eksempel på dette kan være en lærer som spør "Hvem er studentene som fullførte siste oppgave", som kan kalle en funksjon som heter `get_completed(student_name: string, assignment: int, current_status: string)`

**Lage strukturert data**  
Brukere kan ta en tekstblokk eller CSV og bruke LLM til å hente ut viktig informasjon fra den. For eksempel kan en student gjøre om en Wikipedia-artikkel om fredsavtaler for å lage AI-flashkort. Dette kan gjøres ved å bruke en funksjon som heter `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Lage ditt første funksjonskall

Prosessen med å lage et funksjonskall består av tre hovedsteg:
1. Kall Chat Completions API med en liste over funksjonene dine og en brukermelding
2. Les modellens svar for å utføre en handling, for eksempel kjøre en funksjon eller et API-kall
3. Gjør et nytt kall til Chat Completions API med svaret fra funksjonen din for å bruke denne informasjonen til å lage et svar til brukeren.


### Elementer i et funksjonskall

#### Brukerens inndata

Første steg er å lage en brukermelding. Dette kan settes dynamisk ved å hente verdien fra et tekstfelt, eller du kan sette en verdi her. Hvis dette er første gang du jobber med Chat Completions API, må vi definere `role` og `content` for meldingen.

`role` kan være enten `system` (lager regler), `assistant` (modellen) eller `user` (sluttbrukeren). For funksjonskall setter vi dette til `user` og gir et eksempelspørsmål.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Lage funksjoner.

Nå skal vi definere en funksjon og parameterne til den funksjonen. Vi bruker bare én funksjon her som heter `search_courses`, men du kan lage flere funksjoner.

**Viktig**: Funksjoner blir lagt til i systemmeldingen til LLM-en og vil telle med i antall tilgjengelige tokens du har.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definisjoner**

`name` - Navnet på funksjonen som vi ønsker skal bli kalt.

`description` - Dette er en beskrivelse av hvordan funksjonen fungerer. Her er det viktig å være spesifikk og tydelig.

`parameters` - En liste over verdier og formatet du ønsker at modellen skal bruke i sitt svar.

`type` - Datatypen som egenskapene vil bli lagret i.

`properties` - Liste over de spesifikke verdiene som modellen vil bruke i sitt svar.

`name` - Navnet på egenskapen som modellen vil bruke i sitt formaterte svar.

`type` - Datatypen til denne egenskapen.

`description` - Beskrivelse av den spesifikke egenskapen.

**Valgfritt**

`required` - Påkrevd egenskap for at funksjonskallet skal kunne fullføres


### Lage funksjonskallet
Etter at vi har definert en funksjon, må vi nå inkludere den i kallet til Chat Completion API. Dette gjør vi ved å legge til `functions` i forespørselen. I dette tilfellet `functions=functions`.

Det finnes også et alternativ for å sette `function_call` til `auto`. Dette betyr at vi lar LLM-en bestemme hvilken funksjon som skal kalles basert på brukermeldingen, i stedet for å tildele det selv.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nå kan vi se på svaret og hvordan det er formatert:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Du kan se at navnet på funksjonen blir kalt, og fra brukermeldingen klarte LLM å finne dataene som passer til argumentene for funksjonen.


## 3. Integrere funksjonskall i en applikasjon.

Etter at vi har testet det formaterte svaret fra LLM, kan vi nå integrere dette i en applikasjon.

### Håndtere flyten

For å integrere dette i applikasjonen vår, la oss følge disse stegene:

Først gjør vi et kall til Open AI-tjenestene og lagrer meldingen i en variabel som heter `response_message`.


In [ ]:
response_message = response.choices[0].message

Nå skal vi definere funksjonen som vil kalle Microsoft Learn API for å hente en liste over kurs:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Som en god praksis vil vi deretter se om modellen ønsker å kalle en funksjon. Etter det vil vi opprette en av de tilgjengelige funksjonene og matche den til funksjonen som blir kalt.
Deretter tar vi argumentene til funksjonen og kobler dem til argumentene fra LLM.

Til slutt legger vi til meldingen om funksjonskallet og verdiene som ble returnert av `search_courses`-meldingen. Dette gir LLM all informasjonen den trenger for å
svare brukeren med naturlig språk.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodeutfordring

Godt jobbet! For å fortsette læringen om Azure Open AI Function Calling kan du bygge: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Flere parametere for funksjonen som kan hjelpe brukere med å finne flere kurs. Du finner tilgjengelige API-parametere her:
 - Lag en ny funksjonskall som tar imot mer informasjon fra brukeren, som for eksempel morsmål
 - Lag feilhåndtering dersom funksjonskallet og/eller API-kallet ikke returnerer noen relevante kurs



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på sitt originale språk skal anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell, menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
